# Обучение модели

## Подключение зависимостей

In [1]:
%pip install torch torchvision torchaudio
%pip install pandas
%pip install sentencepiece
%pip install transformers
%pip install datasets
%pip install seqeval


DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/opt/homebrew/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installa

## Преподготовка

In [2]:
import pandas as pd
from datasets import Dataset
from datasets import load_metric
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
import torch
import numpy as np

/opt/homebrew/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
path = "../data/combined/"


In [19]:
train_df = pd.read_json(path + "onto_bert_train.json")
test_df = pd.read_json(path + "onto_bert_test.json")

In [20]:
train_df

,tokens,ner_tags
0,"[i, want, to, fly, from, boston, at, 838, am, ...","[0, 0, 0, 0, 0, fromloc.city_name, 0, time, ti..."
1,"[what, flights, are, available, from, pittsbur...","[0, 0, 0, 0, 0, fromloc.city_name, 0, toloc.ci..."
2,"[what, is, the, arrival, time, in, san, franci...","[0, 0, 0, flight_time, flight_time, 0, fromloc..."
3,"[cheapest, airfare, from, tacoma, to, orlando]","[cost_relative, 0, 0, fromloc.city_name, 0, to..."
4,"[round, trip, fares, from, pittsburgh, to, phi...","[round_trip, round_trip, 0, 0, fromloc.city_na..."
...,...,...
10195,[Biersack],[last_name]
10196,"[Four, persons, will, be, dining, in, our, party]","[people, 0, 0, 0, 0, 0, 0, 0]"
10197,"[in, 12, more, days]","[date, date, date, date]"
10198,[18],[time]


In [21]:
test_df

,tokens,ner_tags
0,"[i, would, like, to, find, a, flight, from, ch...","[0, 0, 0, 0, 0, 0, 0, 0, fromloc.city_name, 0,..."
1,"[on, april, first, i, need, a, ticket, from, t...","[0, date, date, 0, 0, 0, 0, 0, fromloc.city_na..."
2,"[on, april, first, i, need, a, flight, going, ...","[0, date, date, 0, 0, 0, 0, 0, 0, fromloc.city..."
3,"[i, would, like, a, flight, traveling, one, wa...","[0, 0, 0, 0, 0, 0, round_trip, round_trip, 0, ..."
4,"[i, would, like, a, flight, from, orlando, to,...","[0, 0, 0, 0, 0, 0, fromloc.city_name, 0, toloc..."
...,...,...
3264,"[The, Cazaree, party, of, two]","[0, last_name, 0, 0, people]"
3265,"[Use, the, name, Anne, Pegg]","[0, 0, 0, first_name, last_name]"
3266,"[Book, under, Elanor, Dinkel]","[0, 0, first_name, last_name]"
3267,"[Karla, Trevizo, i, don't, remember, the, phon...","[first_name, last_name, 0, 0, 0, 0, 0, 0]"


In [3]:
entity_classes = [
 '0',
 'days_code',
 'last_name',
 'airline_name',
 'round_trip',
 'flight_number',
 'fromloc.state_code',
 'toloc.country_name',
 'connect',
 'transport_type',
 'airport_code',
 'toloc.city_name',
 'people',
 'fromloc.city_name',
 'cost_relative',
 'flight_time',
 'toloc.airport_name',
 'city_name',
 'mod',
 'or',
 'date',
 'toloc.airport_code',
 'flight_stop',
 'meal',
 'meal_description',
 'flight_mod',
 'fromloc.state_name',
 'flight_days',
 'airport_name',
 'meal_code',
 'state_name',
 'time',
 'fromloc.airport_code',
 'state_code',
 'aircraft_code',
 'restriction_code',
 'stoploc.city_name',
 'fare_amount',
 'fromloc.airport_name',
 'economy',
 'first_name',
 'airline_code',
 'class_type',
 'time_relative',
 'toloc.state_name',
 'toloc.state_code',
 'fare_basis_code',
 'period_mod']

In [26]:
labels_to_ids = {k: v for v, k in enumerate(entity_classes)}
ids_to_labels = {v: k for v, k in enumerate(entity_classes)}
 
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

task = 'ner'
model_checkpoint = "distilbert-base-uncased"
batch_size = 16
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [28]:
def tokenize_and_align_labels(examples):
    label_all_tokens = True
    tokenized_inputs = tokenizer(list(examples["tokens"]), truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif label[word_idx] == '0':
                label_ids.append(0)
            elif word_idx != previous_word_idx:
                label_ids.append(labels_to_ids[label[word_idx]])
            else:
                label_ids.append(labels_to_ids[label[word_idx]] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)
        
    tokenized_inputs["labels"] = labels
    return tokenized_inputs


train_tokenized_datasets = train_dataset.map(tokenize_and_align_labels, batched=True)
test_tokenized_datasets = test_dataset.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/11 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

## Обучение

In [35]:
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(entity_classes))

args = TrainingArguments(
    f"test-{task}",
    evaluation_strategy = "epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=1e-5,
)

data_collator = DataCollatorForTokenClassification(tokenizer)
metric = load_metric("seqeval")


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [[entity_classes[p] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    true_labels = [[entity_classes[l] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {"precision": results["overall_precision"], "recall": results["overall_recall"], "f1": results["overall_f1"], "accuracy": results["overall_accuracy"]}
    
trainer = Trainer(
    model,
    args,
    train_dataset=train_tokenized_datasets,
    eval_dataset=test_tokenized_datasets,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()
trainer.evaluate()
trainer.save_model('udbert-ner_many.model')

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
 

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.224400,0.120373,0.912746,0.903400,0.908049,0.970308
2,0.071200,0.098094,0.928958,0.929675,0.929316,0.976153
3,0.047000,0.097737,0.928400,0.936920,0.932641,0.976934
4,0.022900,0.102973,0.937086,0.942427,0.939749,0.978497
5,0.014400,0.104287,0.937128,0.943103,0.940106,0.978778


Saving model checkpoint to test-ner/checkpoint-500
Configuration saved in test-ner/checkpoint-500/config.json
Model weights saved in test-ner/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-ner/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-ner/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: tokens, ner_tags, __index_level_0__. If tokens, ner_tags, __index_level_0__ are not expected by `DistilBertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3269
  Batch size = 16
/usr/local/lib/python3.6/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 0 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.6/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning:

Saving model checkpoint to udbert-ner_many.model
Configuration saved in udbert-ner_many.model/config.json
Model weights saved in udbert-ner_many.model/pytorch_model.bin
tokenizer config file saved in udbert-ner_many.model/tokenizer_config.json
Special tokens file saved in udbert-ner_many.model/special_tokens_map.json
